# Numpy

Numpy is a very powerful set of modules for doing numerical operations and manipulations in python. More specifically, numpy is all about multi-dimensional arrays. Also, numpy can replace panda dataframes.

### Array creation

First, let's look at creating some arrays with numpy.

In [ ]:
# Array creating, zeros, ones, empty, full, array
import numpy

one = numpy.ones(5, int)
print(one)

zero = numpy.zeros(5, float)
print(zero)

empty = numpy.empty([5,2])
print(empty)

full = numpy.full([2, 2], 5, int)
print(full)

arange = numpy.arange(10)
print(arange)

arr = numpy.array([[1, 2, 3], [4, 5, 6]], int)
print(arr)

Just like other python classes, numpy arrays have several attributes that we can look at.

In [ ]:
# Array attributes dtype, shape, size
print(arr.dtype)
print(arr.shape)
print(arr.size)

### Indexing

Okay, but how do I get information from or put information into my array?

In [ ]:
# Indexing an array
arr[0, 0] = 100
print(arr[0, 0])

arr[:, 0] = 101
print(arr)

arr[(0, 1, 0), (0, 1, 2)] = 9
print(arr)

arr[(0, 1, 0), 0] = 10
print(arr)

print(arr[0, :], type(arr[0, :]))

arr[-1, -2:] = 50
print(arr)

print(numpy.arange(10)[::-2])

### Where

We can also perform operations on the entire array without having to loop through it. Let's find out which numbers between 0 and 99 are divisible by 7.

In [ ]:
# Create an arange array and check %7==0
A = numpy.arange(100)
mask = (A % 7) == 0
print(mask)

where = numpy.where(mask)
print(where)

A = A.reshape(2, -1)
print(A.shape)
where = numpy.where((A % 7) == 0)
print(where)

### Reshaping

We can also easily create a 1D array with either the `flatten` or `ravel` methods. The difference is that flatten returns a copy of the array while ravel creates a new *view* of the array (it doesn't touch the data but defines how to take indexing values and map them to data). This means that you can still change data in the original view of a raveled array and see those changes reflected in raveled view.

In [ ]:
A = numpy.arange(100).reshape(10, 10)
A_flat = A.flatten()
A_ravel = A.ravel()
A[0, :10] = 0
print(A_flat)
print("")
print(A_ravel)

There is the question of how does numpy decide how to flatten data. The answer is that there are two standards, C (join rows) and Fortran-like (join columns). Numpy defaults to C but you can specify either.

### Loading from text file

Now let's try something more pratical. Let's say we have a set of genes and enhancers and we want to know all of the enhancer-gene pairs that fall within 1000 bases.

In [ ]:
# Load gene and enhancer coordinates and find distances
genes = numpy.loadtxt("genes.txt", dtype=int)
enhancers = numpy.loadtxt("enhancers.txt", dtype=int)

### Broadcasting

Now lets introduce the concept of *broadcasting*. Broadcasting is implicitly altering the shape(s) of arrays to match.

In [ ]:
dist = numpy.abs(genes.reshape(-1, 1) - enhancers.reshape(1, -1))
where = numpy.where(dist <= 1000)
print(where)

### Binary search

Can we use numpy to more easily solve the binary search problem? Yes! The method `searchsorted` actually uses a binary search algorithm.

In [ ]:
# Let's find the position of enhancer8
pos = enhancers[8]
index = numpy.searchsorted(genes, pos)
print(index)
print(pos - genes[10], genes[11] - pos)

In fact, we can actually solve the problem of gene-enhancer pairs <= 1000bp without doing all of the comparisons. Let's see how:

In [ ]:
gene_starts = numpy.searchsorted(genes + 1000, enhancers)
gene_ends = numpy.searchsorted(genes - 1000, enhancers)
for i in range(enhancers.shape[0]):
    print(i, list(range(gene_starts[i], gene_ends[i])))


### Structured arrays

But, what if I have different types of data in my file, not just all ints, or all floats? Can I still load it? You sure can.

In [ ]:
a=numpy.loadtxt("test.txt", dtype=numpy.dtype([('ints', int), ('floats', float), ("chars", "<U1")]))
print(a)
print(a['ints'])
print(a[2])

### npy and npz files

Can I avoid having to convert back and forth from text to integers, floats? Why, yes, you can. Numpy allows you to save arrays exactly as they are encoded so it's fast to save and load them. The only downside is that you can't easily look at them with a text editor. You're also not limited to a single array

In [ ]:
numpy.save("test.npy", a)
b = numpy.load("test.npy")
print(b)
numpy.savez("test.npz",a=b, b=numpy.ones(3,int))
#numpy.savez("test.npz",**{"a":b, "b":numpy.ones(3,int)})
c = numpy.load("test.npz")
print(c["b"])

### uFuncs

One thing that numpy excels at is being crazy fast with math operations on a matrix. That's because it uses compiled code to perform the operations, rather than sluggish Python. The result is that your python can approach C speeds if you are using numpy for everything and avoiding loops. These compiled functions are called *uFuncs* and are accessed with standard math operators. However, the one requirement for numpy to use a uFunc is that the arrays need to be of the same data type.

In [ ]:
from timeit import timeit
a = numpy.ones((5,5),int)
b = numpy.zeros((5,5),int)
c = numpy.zeros((5,5),float)
%timeit a + b
%timeit a + c

### Image data

Numpy also provides a nice platform for working with things like image data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image = plt.imread("lena.png")
print(image.shape, image.dtype)
plt.imshow(image)


In [ ]:
from scipy.ndimage import gaussian_filter
image2 = gaussian_filter(image, 2.0)
plt.imshow(image2)

In [ ]:
# Let's make our own gaussian kernel
sigma = 2.0
n = int(numpy.ceil(sigma * 4))
ax = numpy.linspace(-n, n, n * 2 + 1)
kernel1D = numpy.exp(-0.5 * ax ** 2 / sigma ** 2)
kernel = kernel1D.reshape(1, -1) * kernel1D.reshape(-1, 1)
kernel[numpy.where((ax.reshape(1, -1) ** 2 + ax.reshape(-1, 1) ** 2) ** 0.5 > 4 * sigma)] = 0
kernel /= numpy.sum(kernel)
print(kernel)

In [ ]:
image3 = numpy.zeros_like(image)
total = numpy.zeros((image.shape[0], image.shape[1], 1), image.dtype)
for x in range(-n, n + 1):
    for y in range(-n, n + 1):
        image3[max(0, x):(image.shape[0] + min(0, x)), max(0, y):(image.shape[1] + min(0, y)), :] += (
            image[max(0, -x):(image.shape[0] + min(0, -x)), max(0, -y):(image.shape[1] + min(0, -y)), :] *
            kernel[x + n, y + n])
        total[max(0, x):(image.shape[0] + min(0, x)), max(0, y):(image.shape[1] + min(0, y)), 0] += kernel[x + n, y + n]
image3 /= total
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(12,25, forward=True)
ax[0].imshow(image2)
ax[1].imshow(image3)
plt.show()